<a href="https://colab.research.google.com/github/advcloud/colabtest/blob/main/mediatest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#claireye.com.tw
st = 0
intv = 3
stop1 = 300


In [2]:
!pip install git+https://github.com/advcloud/colablibm.git
!git clone https://github.com/advcloud/colablib1.git
%cd colablib1
import libmedia

  Cloning https://github.com/advcloud/colablibm.git to /tmp/pip-req-build-izbtwlzw
  Running command git clone -q https://github.com/advcloud/colablibm.git /tmp/pip-req-build-izbtwlzw
     |████████████████████████████████| 32.8 MB 270 kB/s 
  Created wheel for colablibm: filename=colablibm-1.0-py3-none-any.whl size=1351 sha256=b781b77379cef8cad85420b4065023cd637d6b8cc6286517061a2a5dc7e8aa11
  Stored in directory: /tmp/pip-ephem-wheel-cache-il0zz_mj/wheels/f9/a7/b3/05fe44ff04c2d7e572c3ebf5d58ae7f5f0d53edde1d4aba60b
Successfully built colablibm
Cloning into 'colablib1'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.
/content/colablib1


In [3]:
from google.colab import files

uploaded = files.upload()
video_path = list(uploaded.keys())[0]

Saving dance.mp4 to dance.mp4


In [4]:
if libmedia.os.path.isdir('images'):
    libmedia.shutil.rmtree('images')
 
libmedia.os.makedirs('images', exist_ok=True)

libmedia.video_2_images(video_path,st,intv,stop1)

In [5]:
files=[]
for name in sorted(libmedia.glob.glob('./images/*.png')):
    files.append(name)
images = {name: libmedia.cv2.imread(name) for name in files}  
for name, image in images.items():
  # Convert the BGR image to RGB and process it with MediaPipe Pose.
  results = libmedia.holistic.process(libmedia.cv2.cvtColor(image, libmedia.cv2.COLOR_BGR2RGB))
 
  # Draw pose landmarks.
  annotated_image = image.copy()
  libmedia.mp_drawing.draw_landmarks(annotated_image, results.left_hand_landmarks, libmedia.mp_holistic.HAND_CONNECTIONS)
  libmedia.mp_drawing.draw_landmarks(annotated_image, results.right_hand_landmarks, libmedia.mp_holistic.HAND_CONNECTIONS)
  libmedia.mp_drawing.draw_landmarks(
      image=annotated_image, 
      landmark_list=results.face_landmarks, 
      connections=libmedia.mp_holistic.FACEMESH_CONTOURS,
      landmark_drawing_spec=libmedia.drawing_spec,
      connection_drawing_spec=libmedia.drawing_spec)
  libmedia.mp_drawing.draw_landmarks(
      image=annotated_image, 
      landmark_list=results.pose_landmarks, 
      connections=libmedia.mp_holistic.POSE_CONNECTIONS,
      landmark_drawing_spec=libmedia.drawing_spec,
      connection_drawing_spec=libmedia.drawing_spec)
  libmedia.cv2.imwrite(name, annotated_image)  

In [6]:
if libmedia.os.path.exists('./output.mp4'):
   libmedia.os.remove('./output.mp4')
 
! ffmpeg -r 10 -i images/%6d.png\
               -vcodec libx264 -pix_fmt yuv420p output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [7]:
mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + libmedia.b64encode(mp4).decode()
libmedia.HTML(f"""
<video width="85%" height="85%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")